In [1]:
import numpy as np

In [2]:
def generateBitstrings(n):
    bitstrings = []
    for i in range(2**n):
        bitstrings.append(f'{i:0{n}b}')
    return bitstrings

# print(generateBitstrings(3))

In [3]:
def addBitstrings(x, y):
    z = []
    for i, bit in enumerate(x):
        z.append(str((int(bit) + int(y[i])) % 2))
    return ''.join(z)

# print(addBitstrings('0011', '0101'))

In [4]:
# def invertBitstring(x):
#     y = []
#     for bit in x:
#         y.append('1' if bit == '0' else '0')
#     return ''.join(y)

# print(invertBitstring('0011'))

In [5]:
def generateF(bitstrings, s):
    f = {}
    for x in bitstrings:
        if x not in f:
            y = x if x[-1] == '0' else addBitstrings(x, s)
            f[y] = y
            f[addBitstrings(y, s)] = y
    return f

# print(generateF(generateBitstrings(3), '111'))

In [6]:
def hammingDistance(x, y):
    dist = 0
    for i, bit in enumerate(x):
        if bit != y[i]:
            dist += 1
    return dist

# print(hammingDistance('0011', '0101'))

In [7]:
def bitstringToVector(x):
    v = np.zeros(2**len(x))
    v[int(x, 2)] = 1
    return v

# print(bitstringToVector('101'))

In [8]:
def hamiltonian(bitstrings, f):
    n = len(bitstrings[0])
    outerSum = np.zeros((2**(2*n), 2**(2*n)))
    for x in bitstrings:
        v = bitstringToVector(x)
        innerSum = np.zeros((2**n, 2**n))
        for y in bitstrings:
            w = bitstringToVector(y)
            innerSum += hammingDistance(y, f[x]) * np.outer(w, w)
        outerSum += np.kron(np.outer(v, v), innerSum)
    return outerSum

# bitstrings = generateBitstrings(2)
# print(bitstrings)
# f = generateF(bitstrings, bitstrings[-1])
# print(f)
# hamiltonian(bitstrings, f)

In [9]:
for n in range(2, 6):
    bitstrings = generateBitstrings(n)
    f = generateF(bitstrings, bitstrings[-1])
    print(hamiltonian(bitstrings, f))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 2. 0. 0.]
 [0. 0. 0. ... 0. 2. 0.]
 [0. 0. 0. ... 0. 0. 3.]]
[[0. 0. 0. ... 0. 0. 0.]
 [

In [23]:
def solveQubo(q, n):
    mins = []
    for i in range(2**(2**(2*n))):
        x = f'{i:0{2**(2*n)}b}'
        v = np.array(list(map(int, list(x))))
        if v.T @ q @ v == 0:
            mins.append(x)
    return mins

bitstrings = generateBitstrings(2)
f = generateF(bitstrings, bitstrings[-1])
q = hamiltonian(bitstrings, f)
results = solveQubo(q, 2)
print(results)

['0000000000000000', '0000000000001000', '0000000000100000', '0000000000101000', '0000001000000000', '0000001000001000', '0000001000100000', '0000001000101000', '1000000000000000', '1000000000001000', '1000000000100000', '1000000000101000', '1000001000000000', '1000001000001000', '1000001000100000', '1000001000101000']


In [37]:
def interpretResults(results, n):
    decodedResults = set()
    zeroes = results[0]
    for result in results:
        if hammingDistance(result, zeroes) == 1:
            index = result[::-1].find('1')
            decodedResults.add(f'{index:0{2*n}b}'[n:])
    return decodedResults

interpretResults(results, 2) #Bug here...should be 11 and 00

{'01', '11'}